# filter

> This module contains all the various filtering options supported.

In [ ]:
#| default_exp filter

In [ ]:
#| export
import numpy as np

from collections import Counter
from squeakily.helpers import flagged_words, get_words

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def _char_rep_ratio(
    doc: str, # document to be analyzed
    char_rep_len: int, # length of character repetition
) -> float:
    """
    Returns the ratio of character repetitions in a document.
    """
    def calc_ngrams(doc, n):
        char_ngrams = [
            doc[i : i + n] for i in range(len(doc) - n + 1)
        ]
        freq_char_ngrams = Counter(char_ngrams)
        return freq_char_ngrams

    freq_char_ngrams = calc_ngrams(
        doc, char_rep_len
    )
    if len(freq_char_ngrams) == 0:
        return 0
    freq_char_ngrams = list(freq_char_ngrams.values())
    freq_char_ngrams = sorted(freq_char_ngrams, reverse=True)
    val_one = len([el for el in freq_char_ngrams if el == 1])
    num_rep_char_ngrams = min(
        int(np.sqrt(len(freq_char_ngrams))),
        len(freq_char_ngrams) - val_one,
    )
    char_rep_ratio = sum(
        freq_char_ngrams[:num_rep_char_ngrams]
    ) / sum(freq_char_ngrams)
    return char_rep_ratio

In [ ]:
#| export
def check_char_repetition(
    document, # document to be analyzed
    char_repetition_len=10, # length of character repetition
    char_repetition_threshold=0.2, # threshold for character repetition
) -> bool: # returns True if document is below threshold
    """
    Checks if the document is below the character repetition threshold.
    """
    char_rep_ratio = _char_rep_ratio(
        document, char_repetition_len
    )
    return char_rep_ratio <= char_repetition_threshold

In [ ]:
test_str = "aaabbbcccddd"
assert check_char_repetition(test_str, char_repetition_len=3, char_repetition_threshold=0.2) == True

test_str = "aaaaaaabbbcccddd"
assert check_char_repetition(test_str, char_repetition_len=3, char_repetition_threshold=0.2) == False

In [ ]:
#| export
def check_exact_match(): pass

In [ ]:
#| export
def _flag_word_ratio(
    doc: str, # document to be analyzed
    flagged_words: list, # list of flagged words
    get_words_func: callable, # function to get words from document
) -> float: # returns ratio of flagged words in document
    """
    Returns the ratio of flagged words in a document.
    """
    words = get_words_func(doc)
    if not words:
        return 0.
    flagged_words_ratio = len(
        [word for word in words if word in flagged_words]
    ) / len(words)
    if flagged_words_ratio > 1.0:
        flagged_words_ratio = 1.0
    return flagged_words_ratio

In [ ]:
#| exporti
def check_flagged_words(
    document: str, # document to be analyzed
    flagged_words: list = flagged_words["en"], # list of flagged words
    flagged_words_threshold: float = 0.1, # threshold for flagged words
    get_words_func: callable = get_words, # function to get words from document
) -> bool: # returns True if document is below threshold
    """
    Checks if a document contains a high percentage of flagged words.
    """
    cond = True
    if flagged_words:
        flagged_words_ratio = _flag_word_ratio(
            document,
            flagged_words,
            get_words_func,
        )
        cond = flagged_words_ratio <= flagged_words_threshold
    return cond

The `check_flagged_words` filter function is purposefully hidden in this documentation as it would show the flagged words directly in the documentation, which might shock some people.

In [ ]:
assert check_flagged_words("test") == True
assert check_flagged_words("cougar") == False

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()